### VERİYİ TANIMA VE VERİYİ HAZIRLAMA

Veri setimizi modellemeye sokmadan önce, verimizi iyi bir şekilde tanımalı, analiz etmeli ve ona göre bir ön hazırlık gerçekleştirmeliyiz. Verimizi hazırlamak için öncelikle gerekli kütüpanelerimizi ekliyoruz. 

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Gerekli kütüphaneleri ekledikten sonra veri setimizi pandas kütüphanesi yardımıyla okuyoruz ve df değişkenine atıyoruz. 

Ayrıca df.info() ile de veri setimiz hakkında detaylı bilgi sahibi oluyoruz. Veri tipleri, null olmayan değerler ve özellikler, bellek kullanımı gibi değerler verilmiş. 

In [2]:
df = pd.read_excel('/content/Elif_Veri_Kumesi.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   İlan No               739 non-null    object 
 1   Birim Fiyat           779 non-null    float64
 2   Yeni Fiyat            7 non-null      object 
 3   İl                    739 non-null    object 
 4   İlçe                  739 non-null    object 
 5   Mahalle               739 non-null    object 
 6   İlan Tarihi           739 non-null    object 
 7   Güncelleme Tarihi     7 non-null      object 
 8   Konut Şekli           739 non-null    object 
 9   Oda Sayısı            739 non-null    object 
 10  M2                    739 non-null    object 
 11  Bulunduğu Kat         693 non-null    object 
 12  Bina Yaşı             739 non-null    object 
 13  Isınma                739 non-null    object 
 14  Kat Sayısı            682 non-null    object 
 15  Tapu                  3

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Birim Fiyat,779.0,2454.246705,1864.885708,7.6923,16.1818,3000.0,3877.15515,6847.8261
tahmini_satış_fiyatı,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df

,İlan No,Birim Fiyat,Yeni Fiyat,İl,İlçe,Mahalle,İlan Tarihi,Güncelleme Tarihi,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Cephe,Yakıt Tipi,tahmini_satış_fiyatı
0,88682-691,3166.666700,NaN,Çanakkale,Merkez,Esenler,29.12.2020,NaN,Daire,2 + 1,90 m2 / 75 m2,Yüksek Giriş,10 Yaşında,Kombi,4 Katlı,NaN,"Güney, Doğu",Doğalgaz,NaN
1,17913-7996,3000.000000,NaN,Çanakkale,Merkez,Esenler,29.12.2020,NaN,Daire,3 + 1,120 m2 / 110 m2,8. Kat,25 Yaşında,Merkezi,8 Katlı,NaN,"Güney, Doğu",Kömür-Odun,NaN
2,17913-7999,3000.000000,NaN,Çanakkale,Merkez,Esenler,29.12.2020,NaN,Daire,2 + 1,95 m2 / 90 m2,2. Kat,25 Yaşında,Kombi,6 Katlı,NaN,"Güney, Batı",Doğalgaz,NaN
3,28849-9186,4172.727300,NaN,Çanakkale,Merkez,Esenler,29.12.2020,NaN,Daire,3 + 1,110 m2 / 90 m2,Yüksek Giriş,10 Yaşında,Kombi,5 Katlı,NaN,Batı,Doğalgaz,NaN
4,68031-693,4450.000000,NaN,Çanakkale,Merkez,Esenler,29.12.2020,NaN,Daire,2 + 1,100 m2 / 90 m2,1. Kat,Sıfır Bina,Merkezi,7 Katlı,Kat İrtifakı,"Kuzey, Güney, Batı",Doğalgaz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,121266-21,11.538500,NaN,Çanakkale,Merkez,Kepez (Cumhuriyet),25.1.2021,NaN,Daire,1 + 1,65 m2 / 60 m2,3. Kat,2 Yaşında,Kombi,4 Katlı,NaN,NaN,Doğalgaz,NaN
775,121266-8,10.833300,NaN,Çanakkale,Merkez,Kepez (Cumhuriyet),25.1.2021,NaN,Daire,1 + 1,60 m2 / 55 m2,3. Kat,4 Yaşında,Merkezi,4 Katlı,NaN,NaN,Doğalgaz,NaN
776,NaN,13.238706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,NaN,16.666700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verimizi gösteren çıktıdan da anlaşılacağı gibi modelimizi eğitirken, ham veri kümemizde bulunan birçok sütuna (özelliğe) ihtiyacımız olmayacaktır. Çünkü ilan tarihi, ilan no, yeni fiyat, il, ilçe, mahalle gibi özellikler modelimizin fiyat tahmini yapmasında kullanılmayacaktır. Bu nedenle de ham veri setimiz işleme tabi tutulmadan önce gerekli olan ön işlemleri yapmamız gerekiyor.

In [5]:
df.drop("İlan No", axis = 1, inplace = True)
df.drop("Yeni Fiyat", axis = 1, inplace = True)
df.drop("İl", axis = 1, inplace = True)
df.drop("İlçe", axis = 1, inplace = True)
df.drop("Mahalle", axis = 1, inplace = True)
df.drop("İlan Tarihi", axis = 1, inplace = True)
df.drop("Güncelleme Tarihi", axis = 1, inplace = True)
df.drop("Cephe", axis = 1, inplace = True)
df.drop("tahmini_satış_fiyatı", axis = 1, inplace = True)

Şimdi ise geri kalan verilerimizdeki düzeltmeleri yapmamız gerekiyor. Örneğin veri setimizde m2 de alan ve net metre kare olarak yazılmış. Bunu düzgünce ayırıp gereksiz olan m2 yazılarını silmeliyiz. Ayrıca iki tane m2 değeri olduğundan birini silmemiz daha iyi olacaktır.  

In [6]:
df["M2"] = df.M2.str[:3]

Bina yaşı kısmında ise 10 yaşında 25 yaşında gibi veriler var bunları daha doğru yorumlayabilmek açısından yaşında kısımlarını siliyoruz. Ayrıca sıfır bina şeklinde yazanları ise 0 şeklinde değiştirmeliyiz. Bunları gerçekleştirmek için aşağıdaki hücreyi yazıyoruz. Bu yaptığım düzenlemeleri aynı şekilde Kat Sayısı, ve Bulunduğu Kat ifadelerine de uyguluyorum. Böylece bu değerleri sayısal olarak kullanabileceğiz.

In [7]:
df.loc[df['Bina Yaşı'] == "Sıfır Bina", "Bina Yaşı"] = "0 Yaşında"
df["Bina Yaşı"] = df["Bina Yaşı"].str[:-8]

Daha sonra ise gözden kaçırmış olabileceğim verileri anlamak ve farklı değerler varsa değiştirmek adına value_counts() fonksiyonu ile hepsini inceliyorum. Bulunan 11 tane özelliğe baktığımızda birçoğunda hatalı veriler de var, bu hatalı verileri elimizden geldiğince temizlemeliyiz ki modelimiz daha doğru bir sonuç versin. 

In [8]:
df

,Birim Fiyat,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Yakıt Tipi
0,3166.666700,Daire,2 + 1,90,Yüksek Giriş,10,Kombi,4 Katlı,NaN,Doğalgaz
1,3000.000000,Daire,3 + 1,120,8. Kat,25,Merkezi,8 Katlı,NaN,Kömür-Odun
2,3000.000000,Daire,2 + 1,95,2. Kat,25,Kombi,6 Katlı,NaN,Doğalgaz
3,4172.727300,Daire,3 + 1,110,Yüksek Giriş,10,Kombi,5 Katlı,NaN,Doğalgaz
4,4450.000000,Daire,2 + 1,100,1. Kat,0,Merkezi,7 Katlı,Kat İrtifakı,Doğalgaz
...,...,...,...,...,...,...,...,...,...,...
774,11.538500,Daire,1 + 1,65,3. Kat,2,Kombi,4 Katlı,NaN,Doğalgaz
775,10.833300,Daire,1 + 1,60,3. Kat,4,Merkezi,4 Katlı,NaN,Doğalgaz
776,13.238706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,16.666700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df["Konut Şekli"].value_counts()

Daire          717
Residence        7
Bina             6
Müstakil Ev      5
Villa            2
Yazlık           1
Kooperatif       1
Name: Konut Şekli, dtype: int64

Konut Şekli çıktısında herhangi değiştirilecek bir değer yok. Bu yüzden bir işlem yapmıyoruz.

In [10]:
df["Oda Sayısı"].value_counts()

2 + 1      251
1 + 1      223
3 + 1      196
4 + 1       35
4 + 2        7
1 + 0        6
5 + 1        5
3 + 2        5
5 + 2        3
6 + 1        2
11 + 1       1
15 + 10      1
8 + 3        1
2 + 2        1
30 + 30      1
9 + 3        1
Name: Oda Sayısı, dtype: int64

Oda sayısını ileride daha rahat kullanabilmek için oda sayısı cinsinden yazmamız daha iyi olacaktır, çünkü geneldeki evlerin bir salonları var. Ayrıca 30 odalı bir ev olamayacağından bu gibi değerleri değiştirmeliyiz.

In [11]:
df["Oda Sayısı"] = df["Oda Sayısı"].str[:-3]
df.loc[df['Oda Sayısı'] == "15 +", "Oda Sayısı"] = 2
df.loc[df['Oda Sayısı'] == "30 +", "Oda Sayısı"] = 3

In [12]:
df["M2"].value_counts()

90     47
60     44
55     40
100    38
65     36
       ..
217     1
83      1
93      1
195     1
744     1
Name: M2, Length: 96, dtype: int64

In [13]:
df["Bulunduğu Kat"].value_counts()

3. Kat          146
2. Kat          139
4. Kat          101
Yüksek Giriş     64
1. Kat           57
5. Kat           48
6. Kat           22
Bahçe Katı       15
En Üst Kat       15
7. Kat           15
Giriş Katı       13
Ara Kat          13
Zemin            12
Kot 2             9
Kot 1             6
Çatı Katı         5
Villa Katı        4
8. Kat            3
Teras Katı        2
Yarı Bodrum       2
10. Kat           1
Kot 3             1
Name: Bulunduğu Kat, dtype: int64

Yukarıda görülen kat değerlerini aşağıdaki gibi hepsinin aynı cinsten bulunduğu kat olması için gerekli işlemleri yapıyoruz.

In [14]:
df.loc[df['Bulunduğu Kat'] == "Yüksek Giriş", "Bulunduğu Kat"] = "10. Kat"
df.loc[df['Bulunduğu Kat'] == "Zemin", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "En Üst Kat", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "", "Bulunduğu Kat"] = "3. Kat"
df.loc[df['Bulunduğu Kat'] == "Bahçe Katı", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Giriş Katı", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Teras Katı", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Yüksek Giriş", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Villa Katı", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Çatı", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Ara Kat", "Bulunduğu Kat"] = "3. Kat"
df.loc[df['Bulunduğu Kat'] == "Kot 1", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Kot 2", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Kot 3", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Çatı Katı", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Yarı Bodrum", "Bulunduğu Kat"] = df["Kat Sayısı"]
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].str[:-5]
df["Kat Sayısı"] = df["Kat Sayısı"].str[:-5]

df.loc[df['Isınma'] == "Belirtilmemiş", "Isınma"] = "Kombi"

In [15]:
df["Bina Yaşı"].value_counts()

0     158
1      58
4      55
2      45
3      43
5      38
10     36
15     36
25     32
7      31
8      30
20     27
6      27
9      21
30     16
12     12
13     11
18      9
17      9
16      7
35      6
22      6
28      5
19      3
11      3
24      3
14      3
42      3
21      2
26      2
50      1
23      1
Name: Bina Yaşı, dtype: int64

In [16]:
df["Isınma"].value_counts()

Kombi                  557
Merkezi (Pay Ölçer)     97
Merkezi                 73
Klima                    6
Soba                     4
Kat Kaloriferi           2
Name: Isınma, dtype: int64

In [17]:
df["Kat Sayısı"].value_counts()

4      217
5      152
3      113
6       91
7       57
8       24
2       22
1        2
9        2
10       2
Name: Kat Sayısı, dtype: int64

In [18]:
df["Tapu"].value_counts()

Kat Mülkiyeti    216
Kat İrtifakı      95
Arsa               3
Name: Tapu, dtype: int64

In [19]:
df["Yakıt Tipi"].value_counts()

Doğalgaz      630
Kömür-Odun     16
Elektrik        3
Name: Yakıt Tipi, dtype: int64

Üst kısımdaki hücreleri çalıştırdığımızda: Konut Şekli, Oda Sayısı, Kat Sayısı, M2, Tapu ve Yakıt verilerinde bir hata olmadığını görüyoruz. Fakat Bulunduğu kat verilerinde giriş, çatı, villa gibi değerler var bu değerleri düzeltmemiz gerekiyor. Ayrıca Isınma sutünundaki verilerde belirtilmemiş olan değeri de en çok kullanılan değerle değiştiriyoruz.

In [20]:
df

,Birim Fiyat,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Yakıt Tipi
0,3166.666700,Daire,2,90,10,10,Kombi,4,NaN,Doğalgaz
1,3000.000000,Daire,3,120,8,25,Merkezi,8,NaN,Kömür-Odun
2,3000.000000,Daire,2,95,2,25,Kombi,6,NaN,Doğalgaz
3,4172.727300,Daire,3,110,10,10,Kombi,5,NaN,Doğalgaz
4,4450.000000,Daire,2,100,1,0,Merkezi,7,Kat İrtifakı,Doğalgaz
...,...,...,...,...,...,...,...,...,...,...
774,11.538500,Daire,1,65,3,2,Kombi,4,NaN,Doğalgaz
775,10.833300,Daire,1,60,3,4,Merkezi,4,NaN,Doğalgaz
776,13.238706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,16.666700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Yukarıdaki hücre çalıştırıldığında görüldüğü üzere Tapu alanında bir çok NaN değer var bu değerleri en çok kullanılmış tapu verisi ile yani modu ile değiştirmemiz gerekiyor. Bu Nan değerler tüm sutünlarda olabileceğinden bazı sütunlara bu fonksiyonu uyguluyoruz.

In [21]:
df["Konut Şekli"] = df["Konut Şekli"].fillna(df["Konut Şekli"].mode()[0])
df["Oda Sayısı"] = df["Oda Sayısı"].fillna(df["Oda Sayısı"].mode()[0])
df["Bina Yaşı"] = df["Bina Yaşı"].fillna(df["Bina Yaşı"].mode()[0])
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].fillna(df["Bulunduğu Kat"].mode()[0])
df["Isınma"] = df["Isınma"].fillna(df["Isınma"].mode()[0])
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].fillna(df["Bulunduğu Kat"].mode()[0])
df["Tapu"] = df["Tapu"].fillna(df["Tapu"].mode()[0])
df["Yakıt Tipi"] = df["Yakıt Tipi"].fillna(df["Yakıt Tipi"].mode()[0])

Ve son olarak NaN değerlerimizin tümünü dropna fonksiyonu sayesinde sildik.

In [22]:
df = df.dropna()

### KATEGORİK DEĞERLERİN ETİKETLENMESİ

Burada bulunan kategorik değişkenlerimizi etiketlemek için önce sklearn’deki preprocessing modülünü kullanıyoruz. Sonrasında ise yeni bir LabelEncoder() nesnesi üretiyoruz. Kategorik verilerimiz olan Konut Şekli, Isınma, Tapu ve Yakıt tipini fit_transform ile etiketliyoruz.  

In [23]:
df

,Birim Fiyat,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Yakıt Tipi
0,3166.6667,Daire,2,90,10,10,Kombi,4,Kat Mülkiyeti,Doğalgaz
1,3000.0000,Daire,3,120,8,25,Merkezi,8,Kat Mülkiyeti,Kömür-Odun
2,3000.0000,Daire,2,95,2,25,Kombi,6,Kat Mülkiyeti,Doğalgaz
3,4172.7273,Daire,3,110,10,10,Kombi,5,Kat Mülkiyeti,Doğalgaz
4,4450.0000,Daire,2,100,1,0,Merkezi,7,Kat İrtifakı,Doğalgaz
...,...,...,...,...,...,...,...,...,...,...
771,13.7500,Daire,1,80,10,5,Kombi,6,Kat Mülkiyeti,Doğalgaz
772,10.6250,Daire,2,80,3,0,Kombi,3,Kat Mülkiyeti,Doğalgaz
773,15.0000,Daire,1,40,1,4,Merkezi (Pay Ölçer),5,Kat Mülkiyeti,Doğalgaz
774,11.5385,Daire,1,65,3,2,Kombi,4,Kat Mülkiyeti,Doğalgaz


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

df['Isınma Tipi'] = labelencoder.fit_transform(df['Isınma'])
df.drop("Isınma", axis = 1, inplace = True)

df['Konut Tipi'] = labelencoder.fit_transform(df['Konut Şekli'])
df.drop("Konut Şekli", axis = 1, inplace = True)

df['Yakıt Şekli'] = labelencoder.fit_transform(df['Yakıt Tipi'])
df.drop("Yakıt Tipi", axis = 1, inplace = True)

df['Tapu Tipi'] = labelencoder.fit_transform(df['Tapu'])
df.drop("Tapu", axis = 1, inplace = True)

In [25]:
df

,Birim Fiyat,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Kat Sayısı,Isınma Tipi,Konut Tipi,Yakıt Şekli,Tapu Tipi
0,3166.6667,2,90,10,10,4,2,1,0,1
1,3000.0000,3,120,8,25,8,3,1,2,1
2,3000.0000,2,95,2,25,6,2,1,0,1
3,4172.7273,3,110,10,10,5,2,1,0,1
4,4450.0000,2,100,1,0,7,3,1,0,2
...,...,...,...,...,...,...,...,...,...,...
771,13.7500,1,80,10,5,6,2,1,0,1
772,10.6250,2,80,3,0,3,2,1,0,1
773,15.0000,1,40,1,4,5,4,1,0,1
774,11.5385,1,65,3,2,4,2,1,0,1


Bu işlem sonrası artık tüm verilerimiz numerik hale geldi. Hazırladığımız bu veri setini kaydedip bir sonraki aşamamız olan modellemeye geçebiliriz.

In [26]:
df.to_excel("Veri_Kumesi_Edited.xlsx")